# COEN 140, Homework 3: Linear & Ridge Regression
## Robbie Culkin

In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
train = pd.read_csv('data/crime-train.txt',delimiter='\t')
test = pd.read_csv('data/crime-test.txt',delimiter='\t')
train.head()

,ViolentCrimesPerPop,population,householdsize,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,...,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn
0,0.67,-0.45,-1.85,-1.06,0.67,0.08,-0.85,-0.34,0.68,-0.24,...,-0.23,-0.02,-0.53,-1.08,-0.13,-0.66,-0.41,-0.56,1.26,-0.39
1,0.43,-0.45,-0.27,-0.22,-0.17,-0.34,-0.58,-0.50,-1.57,-0.29,...,-0.23,-0.33,-0.58,0.03,0.22,-0.46,-0.50,-0.11,-0.62,-0.39
2,0.12,-0.14,1.87,0.55,0.04,0.02,-1.19,-0.03,0.68,1.05,...,-0.23,-0.11,-1.51,1.07,0.07,-0.01,-0.41,0.77,0.52,-0.39
3,0.03,-0.38,0.53,-0.28,-0.79,-0.64,-0.35,-0.34,0.46,0.66,...,-0.23,-0.46,0.54,0.58,-0.08,-0.61,-0.23,-0.70,-0.62,-0.39
4,0.14,-0.30,-1.12,-0.74,-0.10,-0.40,-0.30,-0.19,0.68,0.76,...,-0.23,2.10,-0.92,-0.25,0.52,-0.06,-0.50,1.71,-0.27,-0.39


In [3]:
X_train = train.drop('ViolentCrimesPerPop',axis=1)
y_train = train['ViolentCrimesPerPop']
print X_train.shape
print y_train.shape

(1595, 95)
(1595,)


In [4]:
X_test = test.drop('ViolentCrimesPerPop',axis=1)
y_test = test['ViolentCrimesPerPop']
print X_test.shape
print y_test.shape

(399, 95)
(399,)


In [5]:
def rmse(y_true,y_pred):
    y_true = y_true.reshape(y_true.shape[0],1)    
    y_pred = y_pred.reshape(y_pred.shape[0],1)
    
    return (sqrt(np.dot((y_true-y_pred).T,(y_true-y_pred))/y_true.shape[0]))[0][0]

## OLS linear regression, closed form
Perform linear regression directly using the closed form solution. Compute the RMSE value on the training data and test data, respectively. 

In [6]:
class OLS_model():
    
    def fit(self,X_train,y_train):
        X = X_train.copy()
        X['bias'] = np.ones(X.shape[0])
        
        self.w = np.dot(np.dot(np.linalg.inv(np.dot(X.T,X)),X.T),y_train)
        return self
    
    def predict(self,X_test):
        X = X_test.copy()
        X['bias'] = np.ones(X.shape[0])
        
        return np.dot(self.w,X.T)

Evaluate on training data

In [7]:
lr = OLS_model()
lr = lr.fit(X_train,y_train)
y_train_hat = lr.predict(X_train)

print 'Training RMSE:', rmse(y_train.values,y_train_hat)

Training RMSE: 0.127689674218


Evaluate on test data

In [8]:
y_test_hat = lr.predict(X_test)

print 'Test RMSE:', rmse(y_test.values,y_test_hat)

Test RMSE: 0.145834644909


## Ridge Regression, closed form
Perform ridge regression directly using the closed form solution. Use k-fold cross validate (k=5) to select the optimal parameter. Compute the RMSE value on the test data. You can begin by running the solver with λ= 400. Then, cut λ down by a factor of 2 and  run again. Continue the process of cutting λ by a factor of 2 until you have models for 10 values of λ in total.

In [9]:
class RR_model:
    
    def __init__(self,lbda):
        self.lbda = lbda
        
    def fit(self,X_train,y_train):
        X = X_train.copy()
        X['bias'] = np.ones(X.shape[0])
        
        self.w = np.dot(np.dot(np.linalg.inv(np.dot(X.T,X)+lbda*np.identity(X.shape[1])),X.T),y_train)
        return self
    
    def predict(self,X_test):
        X = X_test.copy()
        X['bias'] = np.ones(X.shape[0])
        
        return np.dot(self.w,X.T)

Cross validation function

In [10]:
def cross_val(model,X,y,k):
    total = 0
    for ii in range(k):
        X_trn = pd.concat([X.iloc[:ii*len(X)/k], X.iloc[(ii+1)*len(X)/k:]])
        y_trn = pd.concat([y.iloc[:ii*len(y)/k], y.iloc[(ii+1)*len(y)/k:]])
        
        X_tst = X.iloc[ii*len(X)/k:(ii+1)*len(X)/k].copy()
        y_tst = y.iloc[ii*len(y)/k:(ii+1)*len(y)/k].copy()
        
        m = model.fit(X_trn,y_trn)
        y_tst_hat = m.predict(X_tst)
        
        total += rmse(y_tst.values,y_tst_hat)
    return total/k

Search for optimal lambda

In [11]:
lbda = 400. #initial lambda try, will iterate down from here
k=5       #for k-fold cross validation

for ii in range(10): #try different lambdas
    rr = RR_model(lbda)
    print 'Lambda: ',lbda,'CV RMSE:',cross_val(rr,X_train,y_train,k)
        
    lbda = lbda/2.0

Lambda:  400.0 CV RMSE: 0.149512776191
Lambda:  200.0 CV RMSE: 0.140694018453
Lambda:  100.0 CV RMSE: 0.137276772232
Lambda:  50.0 CV RMSE: 0.136155943928
Lambda:  25.0 CV RMSE: 0.135915859199
Lambda:  12.5 CV RMSE: 0.136022374462
Lambda:  6.25 CV RMSE: 0.136267864941
Lambda:  3.125 CV RMSE: 0.136570956936
Lambda:  1.5625 CV RMSE: 0.136887640732
Lambda:  0.78125 CV RMSE: 0.137176268069


From the above result we can see that the optimal lambda value is 25, as this is where our cross validation RMSE is minimized.

Compute test RMSE with this lambda:

In [12]:
rr = RR_model(25)
rr = rr.fit(X_train,y_train)
y_test_hat = rr.predict(X_test)

print rmse(y_test.values, y_test_hat)

0.145877655881


## OLS Linear Regression, gradient descent
Perform linear regression using the gradient descent algorithm. Compute the RMSE value on the training data and test data, respectively. For  the initial  weights, you can just use Gaussian N(0,  1)random variables. Define “converging” as the change in any coefficient between one iteration and the  next  is  no larger than 10^−5. 

In [13]:
class OLS_model_gd:
    
    def fit(self,X_train,y_train,alpha):
        X = X_train.copy()
        X['bias'] = np.ones(X.shape[0])
        
        np.random.seed(42)
        w = np.random.normal(0,1,size=X.shape[1])
        
        epsilon = 1
        while epsilon > 10e-7:
            w_new = w - alpha/float(X.shape[0])*np.dot((np.dot(X,w) - y_train),X)
            epsilon = numpy.linalg.norm(w_new - w) #euclidian distance
            w = w_new
            
        self.w = w
        return self
    
    def predict(self,X_test):
        X = X_test.copy()
        X['bias'] = np.ones(X.shape[0])
        
        return np.dot(self.w,X.T)

Evaluate on training data

In [14]:
lr = OLS_model_gd()
lr = lr.fit(X_train,y_train,0.001)
y_train_hat = lr.predict(X_train)

print 'Training RMSE:', rmse(y_train.values,y_train_hat)

Training RMSE: 0.131493705072


Evaluate on test data

In [15]:
y_test_hat = lr.predict(X_test)

print 'Test RMSE:', rmse(y_test.values,y_test_hat)

Test RMSE: 0.146381677056


## Ridge Regression, gradient descent
Perform ridge regression using the gradient descent algorithm. Compute the RMSE value on the test data. 

In [16]:
class RR_model_gd:
    
    def __init__(self,lbda):
        self.lbda = lbda
        
    def fit(self,X_train,y_train,alpha):
        X = X_train.copy()
        X['bias'] = np.ones(X.shape[0])
        
        np.random.seed(42)
        w = np.random.normal(0,1,size=X.shape[1])
        
        epsilon = 1
        while epsilon > 10e-7:
            w_new = w - alpha/float(X.shape[0])*(np.dot((np.dot(X,w) - y_train),X) + lbda*np.dot(np.identity(X.shape[1]),w))
            epsilon = numpy.linalg.norm(w_new - w) #euclidian distance
            w = w_new
            
        self.w = w
        return self
    
    def predict(self,X_test):
        X = X_test.copy()
        X['bias'] = np.ones(X.shape[0])
        
        return np.dot(self.w,X.T)

Evaluate on training data

In [17]:
rr = RR_model_gd(lbda=25)
rr = rr.fit(X_train,y_train,0.001)
y_train_hat = rr.predict(X_train)

print 'Training RMSE:', rmse(y_train.values,y_train_hat)

Training RMSE: 0.130148414402


Evaluate on test data

In [18]:
y_test_hat = rr.predict(X_test)

print 'Test RMSE:', rmse(y_test.values,y_test_hat)

Test RMSE: 0.145637493808
